In [1]:
import numpy as np
import pandas as pd
import datacleaning
import warnings

warnings.filterwarnings("ignore")                     #忽略不必要的警告

train = pd.read_csv("D:\\QQ\\NLP-beginer\\task1\\dataset\\train.tsv",sep='\t')
test = pd.read_csv("D:\\QQ\\NLP-beginer\\task1\\dataset\\test.tsv",sep='\t')


In [2]:
from bs4 import BeautifulSoup
import  re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import BOW
import NG
import softmaxregression

test_clean=pd.DataFrame(datacleaning.clean(test))
test_clean.columns=['dataclean']
train_clean=pd.DataFrame(datacleaning.clean(train))
train_clean.columns=['dataclean']

In [3]:
train_df=pd.concat([train,train_clean],axis=1)
test_df=pd.concat([test,test_clean],axis=1)

train_x=train_df['dataclean']
train_y=train_df['Sentiment']

test_x=test_df['dataclean']

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

# 设置分层抽样的参数
n_splits = 1
test_size = 0.3
random_state = 42

# 创建 StratifiedShuffleSplit 对象
stratified_split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

# 进行分层抽样
for train_index, val_index in stratified_split.split(train_x,train_y):
    X_train, X_val = train_x[train_index], train_x[val_index]
    y_train, y_val = train_y[train_index], train_y[val_index]


In [5]:
print(train_x)
print(train_y)

0         a series of escapade demonstrating the adage t...
1         a series of escapade demonstrating the adage t...
2                                                  a series
3                                                         a
4                                                    series
                                ...                        
156055                                             hearst s
156056                             forced avuncular chortle
156057                                    avuncular chortle
156058                                            avuncular
156059                                              chortle
Name: dataclean, Length: 156060, dtype: object
0         1
1         2
2         2
3         2
4         2
         ..
156055    2
156056    1
156057    3
156058    2
156059    2
Name: Sentiment, Length: 156060, dtype: int64


In [6]:
bow=BOW.BagofWords()
df_xb=bow.transform(X_train)

In [7]:
ngram=NG.Ngram(3)
df_xn=ngram.N_Gram(X_train)

In [8]:
softmax_bow=softmaxregression.softmax_regression(5)
softmax_ngram=softmaxregression.softmax_regression(5)

In [9]:
softmax_bow.softmax(df_xb,y_train,X_val,y_val)

In [ ]:
X=df_xb
Y=y_train
epoch=100
batchsize=32
num_classes=5
lr=0.01

In [ ]:
samples, features = X.shape
W = np.zeros((features, num_classes))

for _ in range(epoch):
    indices = np.random.permutation(samples)
    for start in range(0, samples, batchsize):
        end = min(start + batchsize, samples)
        batch_indices = indices[start:end]
        X_batch = X[batch_indices]
        y_batch = Y.values[batch_indices]
        for i in range(len(batch_indices)):
            xi = X_batch[i]
            yi = y_batch[i]
            scores = np.dot(xi, W)
            print(scores)
            exp_scores = np.exp(scores - np.amax(scores, axis=1, keepdims=True))  # 数值稳定性
            probs = exp_scores / np.sum(exp_scores)
            loss_grad = xi.reshape(-1, 1) @ ((probs - (yi == np.arange(num_classes))) / batchsize)
            W -= lr * loss_grad